In [1]:
!pip install flake8
!pip install pycodestyle
!pip install pycodestyle_magic

In [2]:
%load_ext pycodestyle_magic

In [8]:
%flake8_off

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import datetime
import time
import pandas as pd
import numpy as np
from tqdm import tqdm

import re

In [28]:
df_개발 = pd.read_csv('C:/datafile/incruit_개발직군.csv')

df_개발.head()

,직업중분류,직업소분류,code,공고수
0,웹프로그래머,웹개발,17111,666
1,웹프로그래머,ASP,14340,60
2,웹프로그래머,CGI·Perl,14420,38
3,웹프로그래머,HTML·DHTML·mHTML,14610,137
4,웹프로그래머,JSP,14690,199


In [29]:
def get_code_s(job_name_s):
    return df_개발.query(f"직업소분류 == '{job_name_s}'").code.values[0]


In [30]:
cond = {'고용형태' : 1, '경력' : 2, '근무지역' : 3, '학력' : 4, '급여조건' : 5, '우대사항' : 6}

def check_condition(condition): #
    
    i = cond[condition]
    
     # i값에 대해 실행이 되는지 확인
    try:
        driver.find_element(By.XPATH, f'/html/body/div[2]/div[1]/div[1]/div/div[1]/div[1]/div/div[2]/div/div/ul/li[{i}]')
        
    except:
        # i값이 없어 에러가 났다면 None을 return
        return None
    
    # i값에 대해 실행이 잘 되었다면 다음을 수행
    else: 
        #버튼을 클릭해 나오는 페이지가 있는지를 확인해서 있다면 버튼 클릭시 나오는 추가 텍스트를 추출
        try:
            botton = driver.find_element(By.XPATH, f'/html/body/div[2]/div[1]/div[1]/div/div[1]/div[1]/div/div[2]/div/div/ul/li[{i}]/div[2]/div/button')
            botton.click()

            #list형태로 반환이 되므로 join을 해주어야 dataFrame을 만들 때 두세줄씩 추가가 안됨
            result = driver.find_element(By.XPATH, f'/html/body/div[2]/div[1]/div[1]/div/div[1]/div[1]/div/div[2]/div/div/ul/li[{i}]/div[2]/div/div/ul').text.split('\n')
            result = ', '.join(result)
        #버튼페이지가 없다면 텍스트 추출
        except:
            result = driver.find_element(By.XPATH, f'/html/body/div[2]/div[1]/div[1]/div/div[1]/div[1]/div/div[2]/div/div/ul/li[{i}]/div[2]').text
        
        #추출 후 다른곳을 클릭해서 열었던 페이지를 지운다.
        finally:
            button = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div[1]/div/div[1]/div[1]/div/div[2]/div/div/ul/li[4]/div[1]/em')
            button.click()
        
        return result

In [31]:
def image_recruit():
    urllist = []
    
    try:
        els = driver.find_elements(By.TAG_NAME, 'img')
    
    except:
        return None
    
    else:
        for el in els:
            urllist.append(el.get_attribute('src'))
    
        return ', '.join(urllist) if urllist else None

In [32]:
def text_recruit():
    
    txt = driver.find_element(By.XPATH, '/html/body/div').text
    txt = txt.replace('\n', ' ')
    return txt

In [33]:
# 무한로딩이 걸릴 시 n회 추가로 재수행
def loading_pass(url, n):
    success = False
    for i in range(n+1):
        try:
            driver.get(url)
            driver.implicitly_wait(20)
            
        except:
            continue
            
        else:
            success = True
            break
        
    return success

In [34]:
def check_company():
    try:
        driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div[1]/div/div[5]/div/h3')
        
    except:
        return (None, None, None)
    
    else:
        try:
            company_size = driver.find_element(By.XPATH, '//*[@id="local_m03"]/div/div/ul[1]/li[2]/div[2]/em').text
        except:
            company_size = None
            
        try:
            company_field = driver.find_element(By.XPATH, '//*[@id="local_m03"]/div/div/ul[1]/li[3]/div[2]/em').text
        except:
            company_field = None
        
        try:
            company_benefit = driver.find_element(By.XPATH, '//*[@id="local_m03"]/div/div/ul[2]/li[1]/div[2]/em').text
        except:
            company_benefit = None
            
        return (company_size, company_field, company_benefit)

In [39]:
code = get_code_s('DBA')

#긁어올 메인페이지 띄우고 main_url경로 초기화
result = pd.DataFrame(columns=['직무중분류', '직무소분류', '채용공고명', '채용기한', '해시태그','채용공고상세_text', '채용공고상세_img', '채용공고상세_url', '고용형태', '경력', '근무지역', '학력', '급여조건', '우대사항',
                        '회사명', '회사규모', '회사분야', '복리후생', '출처'])
failed_page = pd.DataFrame(columns=['직무중분류', '직무소분류', '공고명', 'url'])

driver.get(f"https://job.incruit.com/jobdb_list/searchjob.asp?occ3={code}")
driver.implicitly_wait(15)
current_main_url = driver.current_url

p = 1 
while True:
    print('-'*10, f'{p}page', '-'*10)
    if p != 1: # page가 1이 아니면
        try:
            loading_pass(current_main_url, 2)

            driver.find_element(By.CLASS_NAME, 'next_n').click()
            driver.implicitly_wait(15)

        #끝 페이지에 도달하게 되면 while문 종료    
        except:
            break

    # 현재 메인 페이지(공고리스트) url저장
    current_main_url = driver.current_url

    # 해당 페이지의 공고 개수

    if p == 1:
        j = 2

    else:
        j = 1

    num = len(driver.find_elements(By.XPATH, f'/html/body/div[2]/div[1]/div/div[2]/div[{j}]/div[2]/ul'))

    for i in tqdm(range(1, num + 1)):

        loading_pass(current_main_url, 2)

        recruit_url = driver.find_element(By.XPATH, f'/html/body/div[2]/div[1]/div/div[2]/div[{j}]/div[2]/ul[{i}]/li/div[2]/div[1]/a').get_attribute('href')
        head = driver.find_element(By.XPATH, f'/html/body/div[2]/div[1]/div/div[2]/div[{j}]/div[2]/ul[{i}]/li/div[2]').text.split('\n')[0][:-4] # 뒤에 스크랩 붙는거 빼주는 용도의 slice
        # 교육과정은 공고에서 제외
        if '교육생' in head:
            continue

        job_tag = driver.find_element(By.XPATH, f'/html/body/div[2]/div[1]/div/div[2]/div[{j}]/div[2]/ul[{i}]/li/div[2]').text.split('\n')[2]
        due_date = driver.find_element(By.XPATH, f'/html/body/div[2]/div[1]/div/div[2]/div[{j}]/div[2]/ul[{i}]/li/div[3]/div[1]/span[1]').text
        c = driver.find_element(By.XPATH, f'/html/body/div[2]/div[1]/div/div[2]/div[{j}]/div[2]/ul[{i}]/li/div[1]').text.split('\n')
        corp = re.sub(r" (관심기업)", '', c[0])

        # 나중에 공백 split 해줌
        if len(c) >= 2:
            corp_tag = driver.find_element(By.XPATH, f'/html/body/div[2]/div[1]/div/div[2]/div[{j}]/div[2]/ul[{i}]/li/div[1]').text.split('\n')[1]
        else:
            corp_tag = ''

        #공고상세 페이지 로딩이 길어질 경우 3번 재시도하고 안되면 해당공고는 건너뛴다.
        success = loading_pass(recruit_url, 3)
        #로딩이 길어져 수집에 실패한 경우 실패 url을 남겨 저장
        if not success:
            failed_new_row = {'직무중분류' : df_개발.query(f'code == {code}')['직업중분류'].values[0], 
                   '직무소분류': df_개발.query(f'code == {code}')['직업소분류'].values[0], 
                   '채용공고명': head, '출처': recruit_url}
            failed_page = pd.concat([failed_page, pd.DataFrame(failed_new_row, index=[0])], ignore_index=True)
            continue

        #공고 없음의 에러이미지가 있다면 넘어간다.
        if driver.find_elements(By.CLASS_NAME, 'error-img'):
            continue

        emptype = check_condition('고용형태')
        career = check_condition('경력')
        location = check_condition('근무지역')
        education = check_condition('학력')
        salary = check_condition('급여조건')
        preferred = check_condition('우대사항')
        time.sleep(0.5)

        company_size, company_field, company_benefit = check_company()

        try:
            body_url = driver.find_element(By.XPATH, '//*[@id="ifrmJobCont"]').get_attribute('src')

        #body에 해당하는 xpath가 존재하지 않으면 내용자체가 없는 공고로 의미 없는 데이터라 생각하여 미수집    
        except:
            continue

        success = loading_pass(body_url, 3)
        if not success:
            failed_new_row = {'직무중분류' : df_개발.query(f'code == {code}')['직업중분류'].values[0], 
                   '직무소분류': df_개발.query(f'code == {code}')['직업소분류'].values[0], 
                   '채용공고명': head, '출처': recruit_url}
            failed_page = pd.concat([failed_page, pd.DataFrame(failed_new_row, index=[0])], ignore_index=True)
            continue

        body_text = text_recruit()

        body_img = image_recruit()

        new_row = {'직무중분류' : df_개발.query(f'code == {code}')['직업중분류'].values[0], 
                   '직무소분류': df_개발.query(f'code == {code}')['직업소분류'].values[0], 
                   '채용공고명': head, '채용기한' : due_date, '해시태그': job_tag,
                   '채용공고상세_text': body_text, '채용공고상세_img' : body_img, '채용공고상세_url' : None,
                   '고용형태' : emptype, '경력' : career, '근무지역': location,
                   '학력' : education, '급여조건' : salary, '우대사항' : preferred,
                   '회사명': corp, '회사규모' : company_size, '회사분야': company_field, 
                   '복리후생' : company_benefit, '출처': recruit_url}

        result = pd.concat([result, pd.DataFrame(new_row, index=[0])], ignore_index=True)

    p += 1
print('----------Finish----------')
driver.quit()


In [13]:
x = 14340

df_개발.query(f'code == {x}')['직업중분류'].values[0]

'웹프로그래머'

In [61]:
#창 띄우기

chrome_options = Options()
chrome_options.add_experimental_option("detach", True) #창을 보면서 작업할 수 있음
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.set_page_load_timeout(20) # 무한로딩 탈출을 위한 장치
# driver.maximize_window()

[WDM] - Downloading:  86%|████████████████████████████████████████████████▏       | 5.41M/6.30M [00:00<00:00, 20.6MB/s]

In [43]:
df_successed, df_failed = crawling(get_code_s('PHP'))

---------- 1page ----------


100%|███████████████████████████████████████████████████████████████████████████████| 60/60 [2:25:21<00:00, 145.36s/it]


---------- 2page ----------


100%|███████████████████████████████████████████████████████████████████████████████| 50/50 [2:23:55<00:00, 172.71s/it]


---------- 3page ----------
----------Finish----------


In [50]:
df_successed.to_csv('C:/Users/horya/git/AIvengers_data/재현/incruit_PHP개발.csv', encoding = 'utf-8-sig')

In [55]:
df_successed, df_failed = crawling(get_code_s('DBA'))

---------- 1page ----------


100%|███████████████████████████████████████████████████████████████████████████████| 60/60 [2:29:15<00:00, 149.26s/it]


---------- 2page ----------


100%|███████████████████████████████████████████████████████████████████████████████| 60/60 [2:45:58<00:00, 165.97s/it]


---------- 3page ----------


 91%|█████████████████████████████████████████████████████████████████████████▋       | 10/11 [29:35<02:57, 177.53s/it]


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[2]/div[1]/div/div[2]/div[1]/div[2]/ul[11]/li/div[2]/div[1]/a"}
  (Session info: chrome=114.0.5735.199)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x010EA813+48355]
	(No symbol) [0x0107C4B1]
	(No symbol) [0x00F85358]
	(No symbol) [0x00FB09A5]
	(No symbol) [0x00FB0B3B]
	(No symbol) [0x00FDE232]
	(No symbol) [0x00FCA784]
	(No symbol) [0x00FDC922]
	(No symbol) [0x00FCA536]
	(No symbol) [0x00FA82DC]
	(No symbol) [0x00FA93DD]
	GetHandleVerifier [0x0134AABD+2539405]
	GetHandleVerifier [0x0138A78F+2800735]
	GetHandleVerifier [0x0138456C+2775612]
	GetHandleVerifier [0x011751E0+616112]
	(No symbol) [0x01085F8C]
	(No symbol) [0x01082328]
	(No symbol) [0x0108240B]
	(No symbol) [0x01074FF7]
	BaseThreadInitThunk [0x75F67D59+25]
	RtlInitializeExceptionChain [0x7744B74B+107]
	RtlClearBits [0x7744B6CF+191]


In [70]:
driver.quit()

In [67]:
driver.find_element(By.XPATH, f'/html/body/div[2]/div[1]/div/div[2]/div[1]/div[2]/ul[11]/li/div[2]/div[1]/a').get_attribute('href')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[2]/div[1]/div/div[2]/div[1]/div[2]/ul[11]/li/div[2]/div[1]/a"}
  (Session info: chrome=114.0.5735.199)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x010EA813+48355]
	(No symbol) [0x0107C4B1]
	(No symbol) [0x00F85358]
	(No symbol) [0x00FB09A5]
	(No symbol) [0x00FB0B3B]
	(No symbol) [0x00FDE232]
	(No symbol) [0x00FCA784]
	(No symbol) [0x00FDC922]
	(No symbol) [0x00FCA536]
	(No symbol) [0x00FA82DC]
	(No symbol) [0x00FA93DD]
	GetHandleVerifier [0x0134AABD+2539405]
	GetHandleVerifier [0x0138A78F+2800735]
	GetHandleVerifier [0x0138456C+2775612]
	GetHandleVerifier [0x011751E0+616112]
	(No symbol) [0x01085F8C]
	(No symbol) [0x01082328]
	(No symbol) [0x0108240B]
	(No symbol) [0x01074FF7]
	BaseThreadInitThunk [0x75F67D59+25]
	RtlInitializeExceptionChain [0x7744B74B+107]
	RtlClearBits [0x7744B6CF+191]


In [69]:
len(driver.find_elements(By.XPATH, f'/html/body/div[2]/div[1]/div/div[2]/div[1]/div[2]/ul'))

10

In [68]:
driver.get('https://job.incruit.com/jobdb_list/searchjob.asp?occ3=16182&page=3')

In [19]:
try:
    driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div[1]/div/div[5]/div/h3').text
except:
    print('None')
else:
    print('ok')

ok


In [22]:
driver.find_element(By.XPATH, '//*[@id="local_m03"]/div/div/ul[2]/li[1]/div[2]/em').text

'주5일근무, 인센티브, 우수사원포상, 장기근속자포상, 식대지원, 교통지원'